<a href="https://colab.research.google.com/github/JuanPablo-Ramos/ProyectoKaggle/blob/main/99%20-%20modelo%20soluci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LightGBM Optimizado

Este notebook implementa:
1. Downcast de tipos para reducir RAM
2. Frequency Encoding de variables categóricas
3. Entrenamiento LightGBM con OOF para importar feature importance
4. Selección de características basada en importancia
5. Retraining en features seleccionados
6. Generación de `submission.csv`
Objetivo: mejorar accuracy sin sacrificar uso de memoria.

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

# Cargar datasets
train = pd.read_csv('train.csv')
test  = pd.read_csv('test.csv')
print('Train shape:', train.shape)
print('Test shape:', test.shape)

Train shape: (692500, 21)
Test shape: (296786, 20)


---
- Esta sección optimiza el uso de memoria del dataset al reducir el tipo de datos numéricos a representaciones más pequeñas cuando es posible, como convertir `int64` a `int32` o `float64` a `float32`.


In [ ]:
# Downcast de tipos numéricos para reducir uso de memoria
for df in (train, test):
    ints = df.select_dtypes(include=['int64']).columns
    floats = df.select_dtypes(include=['float64']).columns
    df[ints] = df[ints].apply(pd.to_numeric, downcast='integer')
    df[floats] = df[floats].apply(pd.to_numeric, downcast='float')
print('Downcast completo')

Downcast completo


---
- Aquí se aplica Frequency Encoding a las columnas categóricas. Este método reemplaza cada valor categórico por la frecuencia con la que aparece en el conjunto de datos de entrenamiento. Esto ayuda al modelo a capturar información sobre la distribución de las categorías.

In [ ]:
# Frequency Encoding de categóricas
TARGET = 'RENDIMIENTO_GLOBAL'
IDCOL = 'ID'
features = [c for c in train.columns if c not in [IDCOL, TARGET]]
cat_cols = train[features].select_dtypes(include=['object']).columns.tolist()
for col in cat_cols:
    freq = train[col].value_counts(normalize=True)
    train[col+'_fe'] = train[col].map(freq)
    test[col+'_fe']  = test[col].map(freq).fillna(0)
fe_cols = [c+'_fe' for c in cat_cols]
features = [c for c in features if c not in cat_cols] + fe_cols
print('Nuevos features count:', len(features))

Nuevos features count: 19


---
- Esta sección realiza el entrenamiento inicial de un modelo LightGBM utilizando validación cruzada estratificada (OOF - Out-of-Fold). Durante este proceso, se calculan las importancias de las características, lo que indica cuánto contribuye cada característica a las predicciones del modelo.

In [ ]:
# OOF LightGBM y cálculo de importancias
X = train[features]
y = train[TARGET]

# Codificar variable objetivo
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_encoded = le.fit_transform(y)

skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
oof_pred = np.zeros(len(train), dtype=int)
imp_vals = np.zeros(len(features), dtype=float)
lgb_params = {
    'objective': 'multiclass', 'num_class': len(le.classes_),
    'learning_rate': 0.05, 'num_leaves': 31,
    'feature_fraction': 0.8, 'bagging_fraction': 0.8, 'bagging_freq': 5,
    'metric': 'multi_logloss', 'verbose': -1
}
for tr_idx, val_idx in skf.split(X, y_encoded):
    dtrain = lgb.Dataset(X.iloc[tr_idx], label=y_encoded[tr_idx])
    dval   = lgb.Dataset(X.iloc[val_idx], label=y_encoded[val_idx])
    model = lgb.train(
        params=lgb_params, train_set=dtrain, num_boost_round=500,
        valid_sets=[dval], callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=100)]
    )
    preds = model.predict(X.iloc[val_idx], num_iteration=model.best_iteration)
    oof_pred[val_idx] = np.argmax(preds, axis=1)
    imp_vals += model.feature_importance(importance_type='gain')
imp_vals /= skf.n_splits
acc = accuracy_score(y_encoded, oof_pred)
print(f'OOF Accuracy inicial: {acc:.4f}')
imp_df = pd.DataFrame({'feature': features, 'importance': imp_vals}).sort_values(by='importance', ascending=False)
imp_df.head(20)

Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[498]	valid_0's multi_logloss: 1.2042
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[500]	valid_0's multi_logloss: 1.20695
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[500]	valid_0's multi_logloss: 1.20619
OOF Accuracy inicial: 0.4313


,feature,importance
7,ESTU_VALORMATRICULAUNIVERSIDAD_fe,517993.999893
5,ESTU_PRGM_ACADEMICO_fe,285942.227938
6,ESTU_PRGM_DEPARTAMENTO_fe,183538.306017
1,coef_1,117558.721625
18,FAMI_EDUCACIONMADRE_fe,84055.794396
15,ESTU_PAGOMATRICULAPROPIO_fe,62647.733063
11,FAMI_EDUCACIONPADRE_fe,60184.119851
2,coef_2,53195.322773
8,ESTU_HORASSEMANATRABAJA_fe,48990.603639
0,PERIODO,41709.453107


---
- Basándose en las importancias calculadas, esta celda selecciona un subconjunto de características que tienen una importancia por encima de la media. Esto ayuda a reducir la dimensionalidad del problema y potencialmente a mejorar el rendimiento y la generalización del modelo.

In [ ]:
# Selección de features: importancia mayor a la media
mean_imp = imp_df['importance'].mean()
selected = imp_df.loc[imp_df['importance'] > mean_imp, 'feature'].tolist()
print(f'Selected {len(selected)} features (imp > {mean_imp:.4f})')

Selected 4 features (imp > 84856.8963)


---
- Finalmente, el modelo LightGBM se re-entrena utilizando únicamente las características seleccionadas. Luego, se generan predicciones en el conjunto de datos de prueba utilizando este modelo re-entrenado. Las predicciones se promedian a través de los folds de validación cruzada, y el resultado final se guarda en un archivo `submission.csv`.

In [ ]:
# Retraining con features seleccionados y predicción en test
X_sel = train[selected]

# Codificar variable objetivo
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_encoded = le.fit_transform(y)


test_pred = np.zeros((len(test), len(y.unique())))
for tr_idx, val_idx in skf.split(X_sel, y_encoded):
    dtrain = lgb.Dataset(X_sel.iloc[tr_idx], label=y_encoded[tr_idx])
    model = lgb.train(lgb_params, dtrain, num_boost_round=model.best_iteration or 500)
    test_pred += model.predict(test[selected], num_iteration=model.best_iteration) / skf.n_splits

final_labels = np.argmax(test_pred, axis=1)

# Decodifica las celdas predichas
final_labels_decoded = le.inverse_transform(final_labels)

submission = pd.DataFrame({
    IDCOL: test[IDCOL],
    TARGET: final_labels_decoded
})
submission.to_csv('submission.csv', index=False)
print("Submission creada: submission.csv")

Submission creada: submission.csv
